# 13-5 事件体系

langchain的事件，通过callbacks进行回调。需要继承BaseCallbackHandler基类(langchain_core.callbacks)。

```python
chain = prompt | llm | StrOutputParser()
```


主要事件如下：

- on_chain_start: 当一个Runnable对象开始调用时触发;
- on_chain_end: 当一个Runnable对象执行结束时触发;
- on_chain_error: 当一个Runnable对象发生错误时触发;
 
- on_llm_start: 当一个llm对象开始调用时触发;(from langchain_ollama.llms import OllamaLLM)
- on_chat_model_start: 当一个ChatModel对象(比如ChatOpenAI/ChatOllama/ChatZhipuAI)开始调用时触发;
- on_llm_end: 当一个ChatModel/llm对象执行结束时触发;
- on_llm_error: 当一个ChatModel/llm对象发生错误时触发;
 
- on_llm_new_token: 在流式调用中(stream()/astream()方法)，每产生一个token触发一次(ChatOllama在invoke()/ainvoke()时也会触发);

回调绑定的两种方式：

1. chain.with_config(callbacks=[LCELEventHandler()])
2. chain.invoke({}, { "callbacks": [LCELEventHandler()] })


In [54]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGenerationChunk, GenerationChunk, LLMResult
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Sequence, TypeVar, Union
from uuid import UUID


# LCEL: langchain expression language(langchain表达式语言)
class LCELEventHandler(BaseCallbackHandler):   
    def on_chain_start(
        self, serialized: dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        print(f"on_chain_start,kwargs:{kwargs}")

    def on_chain_end(self, outputs: dict[str, Any], **kwargs) -> None:
        print(f"on_chain_end,kwargs:{kwargs}")

    def on_chain_error(
        self,
        error: BaseException,
        **kwargs: Any,
    ) -> None:
        print(f"on_chain_error,kwargs:{kwargs}")

    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print(f"on_chat_model_start,kwargs:{kwargs}")

    def on_llm_start(
        self,
        serialized: Dict[str, Any],
        prompts: List[str],
        **kwargs: Any,
    ) -> Any:
        print(f"on_llm_start,kwargs:{kwargs}")
    
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"on_llm_end,kwargs:{kwargs}")

    def on_llm_error(
        self,
        error: BaseException,
        **kwargs: Any,
    ) -> None:
        print(f"on_llm_error,kwargs:{kwargs}")
    
    # 流式传输的时候起作用
    def on_llm_new_token(
        self,
        token: str,
        **kwargs: Any,
    ) -> Any:
        print(f"on_llm_new_token,run_id:{kwargs.get("run_id", None)}")



In [63]:
# ChatPromptTempalte
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama, ChatZhipuAI
from langchain_core.output_parsers import StrOutputParser
import os
import getpass

os.environ["ZHIPUAI_API_KEY"] = getpass.getpass("请输入ZHIPUAI_API_KEY...")

prompt = ChatPromptTemplate.from_template("请你使用中文描述一下LangChain的事件体系？")


# 1. 创建llm对象
# chat_model = ChatOllama(model="llama3")
chat_model = ChatZhipuAI(model="glm-4")

# 2. 创建一个langchain对象
parser = StrOutputParser()
chain = (prompt.with_config({"run_name":"prompt_dxh"}) 
| chat_model.with_config({"run_name":"chat_model_dxh"}) 
| parser.with_config({"run_name":"parser_dxh"}) )

chain = chain.with_config({"callbacks": [LCELEventHandler()], "run_name": "chain_dxh"})

# 1. invoke()调用
# result = chain.invoke({},{"callbacks": [LCELEventHandler()]})
# 2. ainvoke()调用
# result = await chain.ainvoke({})
# 3. stream()调用
# result = chain.stream({})
# for chunk in result:
#     ...
# 4. astream()调用
result = chain.astream({})
async for chunk in result:
    ...

请输入ZHIPUAI_API_KEY... ········


on_chain_start,kwargs:{'run_id': UUID('e8054d67-0161-486c-89ad-fefa6bb934e5'), 'parent_run_id': None, 'tags': [], 'metadata': {}, 'run_type': None, 'name': 'chain_dxh'}
on_chain_start,kwargs:{'run_id': UUID('baff963d-d273-4bd0-bb73-c3f5746b53d3'), 'parent_run_id': UUID('e8054d67-0161-486c-89ad-fefa6bb934e5'), 'tags': ['seq:step:1'], 'metadata': {}, 'run_type': 'prompt', 'name': 'prompt_dxh'}
on_chain_end,kwargs:{'run_id': UUID('baff963d-d273-4bd0-bb73-c3f5746b53d3'), 'parent_run_id': UUID('e8054d67-0161-486c-89ad-fefa6bb934e5'), 'tags': ['seq:step:1']}
on_chat_model_start,kwargs:{'run_id': UUID('f5f8f05d-a85c-4cd7-8cf4-7575204ec15d'), 'parent_run_id': UUID('e8054d67-0161-486c-89ad-fefa6bb934e5'), 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'zhipuai', 'ls_model_type': 'chat', 'ls_model_name': 'glm-4', 'ls_temperature': 0.95}, 'invocation_params': {'zhipuai_api_base': 'https://open.bigmodel.cn/api/paas/v4/chat/completions', '_type': 'zhipuai-chat', 'stop': None}, 'options': {'sto